# GMS5203 - Introduction to Data Science

# Lab 1-1: Data Wrangling (CKD Data)

Workshop Instructions: 

1. Read through the text descriptions at the top of the cell/code blocks
2. Run the code below by selecting the code block and pressing ``Ctrl + Enter``. Note: The preceding code blocks must be run before proceeding to the next block.
2. Think through the guiding questions and points that are raised for the step. What do you observe from the output and what do they mean?

In [1]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip (21.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import os as os

# Open Sourced Datasets

The datasets in this notebook can be obtained from the **UCI Machine Learning Repository**

1. Chronic Kidney Disease Dataset: https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

**References**:

1. Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

# 1. Read in datasets

In [3]:
# data_dir = 'C:\\_Work_Folder\\SMU\\MITB_HealthcareAnalytics\\Lectures\\Datasets\\CKD\\'
# os.chdir(data_dir)
url='https://raw.githubusercontent.com/seanlam74/GMS5203-Lab/main/chronic_kidney_disease_full_unclean.csv'
ckd = pd.read_csv(url)

# 2. Data Wrangling and Exploration

The first step in any data analysis process is inspecting the dataset itself and familiarizing with the data format and presentation. Use the `.head()`, `.column()`, `.info()`, `.describe()` etc methods on the DataFrames to have a look at the values in the columns and how they are presented. 

Key Considerations:

- Are there any missing values? How do you want to treat cases with these values?
- Are there duplicated rows?
- Is the data in a tidy format?
- Which is your target variable, or variable that you are interesting in predicting?
- Are there any outliers? What are the descriptives for each column?
- Need to rescale, recode categorical, aggregate, etc?
- Is the dataset balanced i.e. equal proportions of examples from each class label?

In [4]:
ckd

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48,80,1.02,1,0,?,normal,notpresent,notpresent,121,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7,50,1.02,4,0,?,normal,notpresent,notpresent,?,...,38,6000,?,no,no,no,good,no,no,ckd
2,62,80,1.01,2,3,normal,normal,notpresent,notpresent,423,...,31,7500,?,no,yes,no,poor,no,yes,ckd
3,48,70,1.005,4,0,normal,abnormal,present,notpresent,117,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51,80,1.01,2,0,normal,normal,notpresent,notpresent,106,...,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,52,80,1.025,0,0,normal,normal,notpresent,notpresent,99,...,52,6300,5.3,no,no,no,good,no,no,notckd
404,36,80,1.025,0,0,normal,normal,notpresent,notpresent,85,...,44,5800,6.3,no,no,no,good,no,no,notckd
405,57,80,1.02,0,0,normal,normal,notpresent,notpresent,133,...,46,6600,5.5,no,no,no,good,no,no,notckd
406,43,60,1.025,0,0,normal,normal,notpresent,notpresent,117,...,54,7400,5.4,no,no,no,good,no,no,notckd


In [ ]:
ckd.describe()

## 2.1 Check and Remove rows that are complete duplicates

In [ ]:
ckd[ckd.duplicated(subset=None)].sort_values(by=['age'])

In [ ]:
ckd.drop_duplicates(inplace=True)

In [ ]:
# Data Profiler is useful to understand the data
profile = ProfileReport(ckd, title='CKD', html={'style':{'full_width':True}})

# Use the following commands if running on a local Jupyter NB server
# file = ProfileReport(ckd)
# file.to_file(output_file='output.html')

In [ ]:
profile.to_notebook_iframe()

In [ ]:
# profile.to_file('data description.html')

## 2.2 Check Data Types and Handle Missing Values

Missing values are common in healthcare datasets, where certain entries are missing or are not recorded for various reasons e.g. test was not done. These values can present themselves in the raw dataset in various ways: NA, NIL, ?, None...

As in many areas of analytics, there are multiple ways to deal with missing data

### 2.2.1 Replace with 'NaN' value

You can choose to replace these values with an `NaN` value, which is used by both Pandas and Numpy packages or a string value such as 'NA'. However, note that replacing numeric columns with a string will cause errors and incompatibility fitting these values into machine learning models since it changes the data format and type.

In this example, we will change the missing values: '?' to a `NaN` value. 

In [ ]:
ckd.replace('?',np.nan,inplace=True)

### 2.2.2 Further Data Understanding

You should seek to understand the data types of each column and there missingness. Decisions made now on how to deal with these will affect any analytical computations downstream

In [ ]:
profile = ProfileReport(ckd)
profile.to_notebook_iframe()
# profile.to_file(output_file='data description correct type.html')
# Note that numeric and categorical columns are detected differently now. We have now identified the data types of the columns using the Data Profiler (13 numeric, 7 categorical and 5 boolean)

In [ ]:
ckd.head()

In [ ]:
# based on insights from the Data Profiler, we hypothesize the data types for each columns are as follows
categorical_cols = ['rbc','pc','pcc','ba','appet']
boolean_cols=['htn','dm','cad','pe','ane']
numeric_cols = ['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wbcc','rbcc']

In [ ]:
# Confirm the categories in each of the categorical and boolean cols
print('rbc - categorical')
print(ckd['rbc'].value_counts())
print('pc - categorical')
print(ckd['pc'].value_counts())
print('pcc - categorical')
print(ckd['pcc'].value_counts())
print('ba - categorical')
print(ckd['ba'].value_counts())
print('appet - categorical')
print(ckd['appet'].value_counts())

In [ ]:
print('htn - boolean')
print(ckd['htn'].value_counts())
print('dm - boolean')
print(ckd['dm'].value_counts())
print('cad - boolean')
print(ckd['cad'].value_counts())
print('pe - boolean')
print(ckd['pe'].value_counts())
print('ane - boolean')
print(ckd['ane'].value_counts())

In [ ]:
ckd.info()

In [ ]:
# As all the columns are of string types, we need to change them to the correct data types
ckd[numeric_cols]=ckd[numeric_cols].astype('float')
ckd.info()

In [ ]:
ckd['bgr'].mean()

In [ ]:
ckd['bgr_na']=ckd['bgr']

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(ckd[numeric_cols])
ckd[numeric_cols] = imp.transform(ckd[numeric_cols])
ckd[numeric_cols] = ckd[numeric_cols].astype(np.float32)

In [ ]:
ckd[ckd['bgr_na'].isna()]

In [ ]:
ckd.drop(['bgr_na'],axis=1,inplace=True)

Notice how the missing values have been replaced by mean values of the entire column. There are many other methods available for imputation, such as Multi-Variate Imputation (MICE), K-Nearest Neighbours that can be used.

In [ ]:
ckd[numeric_cols].describe()

##  2.3 Recoding categorical and Boolean variables (e.g. One-Hot Encoding and Label Encoding)

Medical datasets often contain categorical variables, and may sometimes have a large number of categories. These variables can indicate the presence of certain conditions e.g. diabetes, hypertension, abnormal results. Before they can be used with certain machine learning models, they have to be transformed into suitable numeric representation, such as **one-hot encoding** or **label encoding**.

**Label Encoding**: Maps the $n$ categories of the variable into integer representations. This is a convenient way to convert the variable but may lead to misleading results, as the variable may be interpreted as a numeric variable instead.

**One-Hot Encoding**: Converts the $n$ categories of the variable in to $n$ new columns, each using a binary value to represent the $nth$ category. This is a common way of transforming categorical variables but may run into dimensionality issues when used with variables with a large number of categories. 

In [ ]:
# Label encoding for Boolean variables ('Yes' -> True; 'No' -> False)
ckd[boolean_cols] = np.where(ckd[boolean_cols] == 'Yes',True,False)
ckd[boolean_cols].info()

In [ ]:
# We can also encode Categorical columns using label encoding as well, but the labels are ambiguous e.g., rbc: {NaN:-1, 'abnormal':0, 'normal': 1}
ckd_cat_le=ckd.copy()
ckd_cat_le[categorical_cols]=ckd_cat_le[categorical_cols].astype('category')
for col in categorical_cols:
    ckd_cat_le[col] = ckd_cat_le[col].cat.codes
ckd_cat_le

In [ ]:
# One-hot encoding for Categorical variables. REmember to keep a dummy variable to indicate NaN
for col in categorical_cols:
    one_hot = pd.get_dummies(ckd[col], dummy_na=True)
    # retain a column to indicate NaN
    one_hot.columns=one_hot.columns.fillna('NA')
    one_hot.columns = [col + '_' + a for a in one_hot.columns]
    one_hot.columns
    ckd = ckd.join(one_hot)
ckd

###  2.4 Recoding/ Rescaling continuous variables 
1. Recoding age into categorical
2. Normalization of wbcc
3. Standardization of wbcc


In [ ]:
# 1. Recoding age into categorical
# Sometimes needed to study risk factors across different age classes. Pre-elderly is unique class as many chronic diseases are detected in this category. Age classes can be an informative feature as well

def age_groups(series):
    if series < 19:
        return "Children"
    elif 20 <= series < 50:
        return "Adults"
    elif 49 <= series < 64:
        return "Pre-elderly"
    elif 65 <= series:
        return "Elderly"

ckd['Age Group'] = ckd['age'].apply(age_groups)
ckd['Age Group'].value_counts(sort=False)

In [ ]:
ckd['wbcc'].describe()

In [ ]:
# 2. Normalization
# wbcc has a min of 2200, max of 26400. Maybe good to consider standardizing this. 
# Normalization can be used to transform the data within range of 0 to 1. 
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

scaler = MinMaxScaler()
ckd['wbcc_scaled_0_1'] = scaler.fit_transform(ckd[['wbcc']])
print('MinMaxScaler:\n{}'.format(ckd['wbcc_scaled_0_1'].describe()))


# There may be outliers in the original dataset and these may bias the rescaling
# Robust Scaler, which uses the IQR, can be used to normalize the dataset. 
robustscaler = RobustScaler()
ckd['wbcc_robust_scaler'] = robustscaler.fit_transform(ckd[['wbcc']])
print('RobustScaler:\n{}'.format(ckd['wbcc_robust_scaler'].describe()))

# However, sometimes domain knowledge may provide more useful norms 
# The normal number of WBCs in the blood is 4,500 to 11,000 WBCs per microliter (4.5 to 11.0 × 109/L). (https://www.ucsfhealth.org/medical-tests/wbc-count) We can consider standardizing wbcc based on domain knowledge 

ckd['wbcc_domain_scaled'] = (ckd['wbcc']-4500)/(11000-4500)
print('DomainScaler:\n{}'.format(ckd['wbcc_domain_scaled'].describe()))


In [ ]:
# 2. Standardization
# Transforms the features to a standard normal distribution with mean 0 and standard 1
std_scaler = StandardScaler()

ckd['wbcc_z_scaled'] = std_scaler.fit_transform(ckd[['wbcc']])
print('Standard Scaler:\n{}'.format(ckd['wbcc_z_scaled'].describe()))



## 3. Tidy Data - Melt

In [ ]:
url='https://raw.githubusercontent.com/ISSS623-AHA/Lecture---Hands-on-Codes/main/Lecture%202/WHO_Children_Under_5_Death_Causes_World.csv'
childmortality=pd.read_csv(url)
childmortality.head()

In [ ]:
childmortality.drop(columns=['level'],inplace=True)
childmortality=childmortality.melt(id_vars = "year", var_name = "reason", value_name = "count")

In [ ]:
childmortality

In [ ]:
childmortality['reason'].value_counts()